![](./logo.png)*„Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”,
projekt finansowany ze środków Programu Operacyjnego Polska Cyfrowa POPC.03.02.00-00-0001/20*



# Widzenie komputerowe
# Moduł laboratoryjny 1, Laboratoria 1 i 2

## Opis laboratoriów



*   Wprowadzenie do przetwarzania obrazów
*   Wizualizacja obrazów,
*   Reprezentacja obrazów w różnych przestrzeniach barw i dziedzinach,
*   Transformacje jednopunktowe Obraz -> Obraz,
*   Arytmetyka obrazowa i przykładowe zastosowanie,
*   Transformacje geometryczne



## Funkcje pomocnicze

Do wykonania zadań niezbędne jest zaimportowanie bibliotek, wykorzystywanych w skrypcie oraz pobranie danych, na których przetwarzane będą operacje.

W skrypcie wykorzystywane będą dwa zestawy danych:
* obraz Lenna (dostępny pod [linkiem](http://www.lenna.org/)) - jeden z najbardziej popularnych obrazów wykorzystywanych historycznie do kompresji i przetwarzania obrazów,
* "Bug Challenge" - zestaw zdjęć mrówki, zrobione z ostrością ustawioną na co raz dalsze fragmenty od obiektywu (dostępny pod [linkiem](http://grail.cs.washington.edu/projects/photomontage/))

In [1]:
# import niezbędnych bibliotek
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pil
from ipython.display import display

ModuleNotFoundError: No module named 'pil'

In [ ]:
# pobranie niezbędnych bibliotek
!wget -O lena_std.tif http://www.lenna.org/lena_std.tif
!wget -O bug.zip http://grail.cs.washington.edu/projects/photomontage/data/bug.zip && unzip -o bug.zip

Ze względu na problem z wyświetlaniem obrazów przez bibliotekę OpenCV w środowisku Colab, w przypadku korzystania z tej platformy należy skorzystać z funkcji specjalnie do tego przygotowanej.

In [ ]:
def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(PIL.Image.fromarray(a))

# Przestrzenie barw

Cyfrowe przechowywanie obrazów polega na reprezentacji koloru w pewnej dziedzinie, która w pewnym stopniu odzwierciedla to, jak człowiek postrzega światło. Najbardziej intuicyjną dziedziną jest **intensywność**, w której obraz składa się z pikseli ułożonych w postaci macierzy 2D. Każdy z pikseli posiada swoją **wartość intensywności**. Tę wartość można przedstawić na wiele sposobów, np.:

* RGB - jakos 3 wartości określające stopień nasycenia kolorami Red, Green, Blue,
* CMYK - analogicznie, Cyan, Magenta, Yellow, Black (Key color),
* HSV - Hue (odcień koloru), Saturation (nasycenie), Value (lub Brightness - jasność koloru),
* Grayscale - odcień szarości,

Różne przestrzenie barw dostarczają różnych możliwości przetwarzania obrazu. Przykładowo z przestrzeni HSV bezpośrednio możemy wyznaczyć jasność, natomiast korzystająć z Grayscale może być łatwiej wykrywać kontury obiektów na scenie.

Oprócz dziedziny intensywności, obraz można również przetwarzać w dziedzinie **częstotliwości**. Obraz w formie macierzy 2D można traktować jako sygnał 2-wymiarowy, a więc podlega on wszelkim operacjom działającym na takich sygnałach, jak **transformata Fouriera**. Reprezentując obraz w dziedzinie częstotliwości mamy możliwość łatwiejszej detekcji krawędzi, obszarów rozmytych i filtrowania obrazów.

Przestrzeń barw RGB:

![rgb.png](https://upload.wikimedia.org/wikipedia/commons/thumb/8/86/RGB_color_model.svg/256px-RGB_color_model.svg.png)


Przestrzeń barw CYMK:


![cmyk.png](https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/CYMK_color_model.svg/256px-CYMK_color_model.svg.png)

## Wczytanie obrazów i początkowa reprezentacja obrazu

Do najbardziej popularnych bibliotek w **Pythonie** do przetarzania obrazów należą:
* OpenCV
* Pillow

Już na samym początku możemy spotkać pewne różnice w przetwarzaniu obrazów przez te biblioteki. **OpenCV** domyślnie działa na obrazach w formacie **BGR**, natomiast Pillow w formacie **RGB**. **BGR** jest niczym innym jak odwróconą kolejnością kolorów dla każdego piksela (Blue, Green, Red).


In [ ]:
# wczytanie obrazu za pomocą biblioteki opencv
img = cv2.imread('./lena_std.tif', 1)  # 1 - color (BGR), 0 - grayscale, -1 unchanged (np. do wczytania kanału alpha)
img = cv2.resize(img, (256, 256))  # zmiana rozmiaru do 256x256

print('Shape:', img.shape)
print('BGR:', img[0, 0])
print('Obraz wczytany i wyświetlony przez OpenCV\n')
imshow(img)

In [ ]:
# biblioteka opencv domyślnie przetwarza obrazy w formacie BGR
# należy o tym pamiętać przy wykorzystywaniu innych bibliotek jak pillow
# (obie biblioteki wczytują dane do tablicy numpy, a więc możliwa jest wymiana funkcjonalności pomiędzy bibliotekami)
img_pil = np.array(PIL.Image.open('./lena_std.tif'))
img_pil = cv2.resize(img_pil, (256, 256))

print('Shape:', img_pil.shape)
print('RGB:', img_pil[0, 0])
print('\nObraz wczytany przez Pillow i wyświetlony przez OpenCV\n')
imshow(img_pil)

In [ ]:
# dopiero po transformacji RGB -> BGR obraz zostanie wyświetlony prawidłowo
imshow(cv2.cvtColor(img_pil, cv2.COLOR_RGB2BGR))

## Transformacja pomiędzy przestrzeniami barw

Pomiędzy przestrzeniami barw możemy swobodnie przechodzić podczas przetwarzania obrazu. Co więcej, większość bibliotek ma domyślnie zaimplementowane takie mechanizmy przejścia, pomiędzy najpopularniejszymi przestrzeniami.


### RGB - HSV

Zakładając dane wejściowe R, G, B, gdzie $R, G, B \in <0,1>$, transformację z przestrzeni RGB do HSV można przedstawić jako:

$C_{max} = max(R,G,B)$  
$C_{min} = min(R,G,B)$  
$\Delta = C_{max} - C_{min}$  

<br/>

${
H=\left\{
  \begin{array}{ll}
    0{\hspace{0.5cm}\text{dla}\hspace{0.5cm}} \Delta = 0\\
    60 * (\frac{G - B}{\Delta} \mod 6){\hspace{0.5cm}\text{dla}\hspace{0.5cm}} C_{max} = R\\
    60 * (\frac{B - R}{\Delta} + 2){\hspace{0.5cm}\text{dla}\hspace{0.5cm}} C_{max} = G\\
    60 * (\frac{R - G}{\Delta} + 4){\hspace{0.5cm}\text{dla}\hspace{0.5cm}} C_{max} = B
  \end{array}
\right.}$  
<br/>
${
S=\left\{
  \begin{array}{ll}
    0{\hspace{0.5cm}\text{dla}\hspace{0.5cm}} C_{max} = 0\\
    \frac{\Delta}{C_{max}} {\hspace{0.5cm}\text{dla}\hspace{0.5cm}} C_{max} \neq 0
  \end{array}
\right.}$  
<br/>
$V = C_{max}$
<br/>

Uwaga #1: S i V powinny być przeskalowane do wartości z przedziału $<0, 255>$

Uwaga #2: OpenCV przetrzymuje wartość H w zakresie $<0, 180>$, a więc ostatecznie H powinno być podzielone przez 2.

### RGB - Grayscale

Transformację z przestrzeni RGB do Grayscale można przedstawić jako:

$$Gray = 0.2989 * R + 0.5870 * G + 0.1140 * B$$

### Implementacja w OpenCV

OpenCV zawiera gotową funkcję **cvtColor**, która jako pierwszy parametr pobiera obraz do przetworzenia, a jako drugi stałą, określającą typ transformacji (stałe oznaczone przez zmienne np. COLOR_RGB2BGR).

In [ ]:
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
img_luv = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

imshow(np.concatenate([img_rgb, img_hsv, img_luv], 1))
imshow(img_grayscale)

In [ ]:

img_grayscale

### **Zadanie 1**

Zaimplementuj następujące transformacje:
* BGR do RGB,
* BGR do HSV,
* BGR do Grayscale

Wyniki zostaną porównane z wynikami funkcji zawartych w OpenCV.

**Uwaga:**  
Mogą występować różnice implementacyjne pomiędzy tymi samymi transformacjami, które mogą wynikać z błędów numerycznych lub po prostu innego typu zaogrąglania / ucinania wartości, przy transformacji liczby zmiennoprzecinkowej do całkowitoliczbowej.

In [ ]:
def BGR2RGB(img_bgr):
    return img_bgr[:,:,::-1]


def BGR2HSV(img_bgr):
    img_bgr_norm = img_bgr / 255
    shape = img_bgr.shape
    C_max = np.max(img_bgr_norm, axis=2)
    C_min = np.min(img_bgr_norm, axis=2)
    delta = C_max - C_min

    H = np.zeros((shape[0], shape[1]))
    for i in range(shape[0]):
        for j in range(shape[1]):
            r, g, b = (img_bgr_norm[i][j][2], img_bgr_norm[i][j][1], img_bgr_norm[i][j][0])
            C_m = C_max[i][j]
            d = delta[i][j]
            if d == 0:
                H[i][j] = 0
            elif C_m == r:
                H[i][j] = 60 * (((g-b)/d) % 6)
            elif C_m == g:
                H[i][j] = 60 * (((b-r)/d) + 2)
            elif C_m == b:
                H[i][j] = 60 * (((r-g)/d) + 4)
            H[i][j] = round(H[i][j])
    H = H / 2

    
    S = np.zeros((shape[0], shape[1]))
    V = np.zeros((shape[0], shape[1]))
    for i in range(shape[0]):
        for j in range(shape[1]):
            if C_max[i][j] == 0:
                S[i][j] = 0
            else:
                S[i][j] = round((delta[i][j] / C_max[i][j]) * 255)
            V[i][j] = round(C_max[i][j] * 255)
    
    return np.stack((H, S, V), axis=2)


def BGR2Gray(img_bgr):
    shape = img_bgr.shape
    img_gray = np.zeros((shape[0], shape[1]))
    for i in range(shape[0]):
        for j in range(shape[1]):
            # 0.2989 * R + 0.5870 * G + 0.1140 * B
            img_gray[i][j] = img_bgr[i][j][0] * 0.1140 + img_bgr[i][j][1] * 0.5870 + img_bgr[i][j][2] * 0.2989
    return img_gray


img_rgb_2 = BGR2RGB(img)
img_hsv_2 = BGR2HSV(img)
img_grayscale_2 = BGR2Gray(img)

imshow(np.concatenate([img_rgb_2, img_hsv_2], 1))
imshow(img_grayscale_2)

print('\n===\n')
print('BGR2RGB Check:', (img_rgb == img_rgb_2).all())
print('BGR2HSV Check:', np.allclose(img_hsv, img_hsv_2, 1, 0))
print('BGR2Grayscale Check:', np.allclose(img_grayscale, img_grayscale_2, 1, 0))

In [ ]:
img_grayscale

## Własne przestrzenie barw

Przestrzenie barw takie jak RGB czy HSV są standardowymi przestrzeniami, wynikającymi z samej natury światła. Nie ogranicza to jednak przed tworzeniem własnych przestrzeni barw. Poniżej zostały przedstawione metody **pseudokolorowania** - czyli nadania pikselom koloru na podstawie sztucznie przygotowanej przestrzeni barw.

Szczególnie interesującą przestrzenią może wydawać się przestrzeń **Hot**, która dla pikseli o większej intensywności (grayscale) nadaje cieplejszą barwę (kolor żółty).


In [ ]:
img_grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_hot = cv2.applyColorMap(img_grayscale, cv2.COLORMAP_HOT)
img_bone = cv2.applyColorMap(img_grayscale, cv2.COLORMAP_BONE)
img_ocean = cv2.applyColorMap(img_grayscale, cv2.COLORMAP_OCEAN)

imshow(np.concatenate([img, img_hot, img_bone, img_ocean], 1))

Aby zaaplikować swoją własną przestrzeń barw, możemy skorzystać z funkcji w bibliotece OpenCV **LUT** (Lookup Table). W poniższym przykładzie przygotowano 3 przestrzenie barw (lut_1, lut_2, lut_3). Każda z tych tablic jest funkcją mapowania pomiędzy każdą z 256 wartości (uint8) a nową wartością. Funkcję można również zastosować dla obrazów **wielokanałowych**.

* **Lut_1** jest funkcją tożsamościową, dla każdej intensywności piksela zwacana jest ta sama intensywność,
* **Lut_2** dla pierwszych 100 wartości (0-99) zwraca intensywność 255, dla kolejnych 100 (100-199) intensywność 0 i dla przedziału 200-256 intensywność 255. Jest to nic innego niż złożenie progowania obrazu,
* **Lut_3** - tworzy ''kubełki'' po 64 wartości (a więc 256 / 64 = 4 progi).

In [ ]:
lut_1 = np.array(range(256))
lut_2 = np.array([255] * 100 + [0] * 100 + [255]* 56)
lut_3 = 64 * (np.array(range(256)) // 64)

img_lut_1 = cv2.LUT(img_grayscale, lut_1)
img_lut_2 = cv2.LUT(img_grayscale, lut_2)
img_lut_3 = cv2.LUT(img_grayscale, lut_3)

imshow(np.concatenate([img_grayscale, img_lut_1, img_lut_2, img_lut_3], 1))

### Zadanie 2

Dla obrazu Lenna w przestrzeni Grayscale najpierw przekształć go do przestrzeni zawierającej 8 progów (kubełków), a następnie transformuj obraz do przestrzeni **Hot**.

Wyświetl wyniki pośrednie.

In [ ]:
lut_8_colors = 32 * (np.array(range(256)) // 32)

img_lut = cv2.LUT(img_grayscale, lut_8_colors).astype(np.uint8)
imshow(np.concatenate([img_grayscale, img_lut], 1))

img_hot = cv2.applyColorMap(img_lut, cv2.COLORMAP_HOT)
imshow(img_hot)

# Operacje jednopunktowe

Operacją jednopunktową nazywamy taką transformację przekształcającą obraz w inny obraz, dla której wynik poszczególnego piksela zależy od odpowiadającemu mu pikselowi w obrazie wejściowym. Formalnie dowolną operację obrazową możemy zapisać następująco:

$$F : I_{in} \rightarrow I_{out}$$

gdzie:

* F - funkcja przekształcająca,
* $I_{in}$ - dziedzina obrazu wejściowego,
* $I_{out}$ - dziedzdina obrazu wyjściowego

dodatkowo, przy ograniczeniu:

$$I_{out}(x,y) = F(I_{in}(x,y)$$

oznaczającym, że piksel obrazu wyjściowego jest wynikiem działania funkcji F na odpowiadającym mu pikselu obrazu wejściowego, możemy zdefiniować operację jednopunktową.

Przykładowymi operacjami jednopunktowymi są (dla uproszczenia zakładamy, że $i \in <0, 1>$):

* tożsamość: $$F(i) = i$$
* odwrotność: $$F(i) = 1 - i$$
* korekcja gamma: $$F(i, \gamma) = i^\gamma$$


### Funkcje pomocnicze

In [ ]:
def plot_simple(ax):
    ax.set_title('Podstawowe transformacje')
    ax.set_xlabel('Intensywność wejściowa')
    ax.set_ylabel('Intensywność wyjściowa')
    ax.plot(i, identity(i), label='identity')
    ax.plot(i, invert(i), label='invert')
    ax.grid()
    ax.legend()

def plot_gamma(ax):
    ax.set_title('Korekcja gamma dla różnych parametrów')
    ax.set_xlabel('Intensywność wejściowa')
    ax.set_ylabel('Intensywność wyjściowa')
    ax.plot(i, gamma(i, 0.1), label='0.1')
    ax.plot(i, gamma(i, 0.2), label='0.2')
    ax.plot(i, gamma(i, 0.5), label='0.5')
    ax.plot(i, gamma(i, 1.0), label='1.0')
    ax.plot(i, gamma(i, 1.8), label='1.8')
    ax.plot(i, gamma(i, 3.0), label='4.0')
    ax.plot(i, gamma(i, 4.5), label='4.5')
    ax.grid()
    ax.legend()

def plot_l_threshold(ax):
    ax.set_title('Dolne odcięcie dla kolejnych progów')
    ax.set_xlabel('Intensywność wejściowa')
    ax.set_ylabel('Intensywność wyjściowa')
    ax.plot(i, l_threshold(i, 0.1), label='0.1')
    ax.plot(i, l_threshold(i, 0.5), label='0.5')
    ax.plot(i, l_threshold(i, 0.9), label='0.9')
    ax.grid()
    ax.legend()


def plot_u_threshold(ax):
    ax.set_title('Górne odcięcie dla kolejnych progów')
    ax.set_xlabel('Intensywność wejściowa')
    ax.set_ylabel('Intensywność wyjściowa')
    ax.plot(i, u_threshold(i, 0.1), label='0.1')
    ax.plot(i, u_threshold(i, 0.5), label='0.5')
    ax.plot(i, u_threshold(i, 0.9), label='0.9')
    ax.grid()
    ax.legend()


def plot_quad(ax):
    ax.set_title('Funkcja kwadratowa')
    ax.set_xlabel('Intensywność wejściowa')
    ax.set_ylabel('Intensywność wyjściowa')
    ax.plot(i, quad(i, 4.0), label='4.0')
    ax.plot(i, quad(i, 2.0), label='2.0')
    ax.plot(i, quad(i, 1.0), label='1.0')
    ax.grid()
    ax.legend()


def plot_stacked(ax):
    ax.set_title('Złożenie transformacji')
    ax.set_xlabel('Intensywność wejściowa')
    ax.set_ylabel('Intensywność wyjściowa')
    ax.plot(i, u_threshold(gamma(invert(i), 0.3), 0.7), label='threshold(gamma(invert))')
    ax.plot(i, quad(l_threshold(i, 0.4), 3.0), label='quad(threshold)')
    ax.grid()
    ax.legend()

### Transformacje punktowe


Poniżej, znajdują się implementacje operacji **tożsamości, odwrotności i korekcji gamma** oraz ich wizualizacje.

In [ ]:
def identity(i):
    return i


def invert(i):
    return 1.0 - i


def gamma(i, g):
    return i ** g

i = np.arange(0.0, 1.0, 0.01)  # dziedzina obrazu

fig, ax = plt.subplots(1, 2, figsize=(15,5), sharex='all', sharey='all')
axes = plt.gca()
axes.set_xlim([0.0, 1.0])
axes.set_ylim([0.0, 1.0])
plot_simple(ax[0])
plot_gamma(ax[1])
plt.show()

### Zadanie 3

Zdefiniuj następujące transformacje:

* Dolne odcięcie (**l_threshold**) - wartości intensywności wejściowej **poniżej** pewnego progu powinny być sprowadzone do tego właśnie progu,
* Górne odcięcie (**u_threshold**) - wartości intensywności wejściowej **powyżej** pewnego progu powinny być sprowadzone do tego właśnie progu,
* Funkcja kwadratowa (**quad**) - funkcja powinna mieć miejsca zerowe dla intensywności wejściowych 0.0 i 1.0 (parametr **a** powinien jedynie sterować **maksimum** funkcji)

In [ ]:
# todo:
def l_threshold(i, threshold):
    return np.clip(i, a_min=threshold, a_max=None)


# todo:
def u_threshold(i, threshold):
    return np.clip(i, a_min=None, a_max=threshold)


# todo:
def quad(i, a):
    return -a * i**2 + a * i


fig, ax = plt.subplots(2, 2, figsize=(15,15), sharex='all', sharey='all')
axes = plt.gca()
axes.set_xlim([0.0, 1.0])
axes.set_ylim([0.0, 1.0])
plot_l_threshold(ax[0, 0])
plot_u_threshold(ax[0, 1])
plot_quad(ax[1, 0])
plot_stacked(ax[1, 1])
plt.show()

### Funkcje pomocnicze

In [ ]:
def imshow_simple(img_bgr):
    print('\n===')
    print('Tożsamość | Odwrotność\n')
    img_bgr = img_bgr / 255.0
    imshow(np.concatenate([
      identity(img_bgr),
      invert(img_bgr)
    ], 1) * 255.0)

def imshow_gamma(img_bgr):
    print('\n===')
    print('Korekcja Gamma 0.1 | 0.5 | 2.0 | 4.0\n')
    img_bgr = img_bgr / 255.0
    imshow(np.concatenate([
      gamma(img_bgr, 0.1),
      gamma(img_bgr, 0.5),
      gamma(img_bgr, 2.0),
      gamma(img_bgr, 4.0)
    ], 1) * 255.0)

def imshow_l_threshold(img_bgr):
    print('\n===')
    print('Dolne odcięcie 0.3 | 0.5 | 0.9\n')
    img_bgr = img_bgr / 255.0
    imshow(np.concatenate([
      l_threshold(img_bgr, 0.3),
      l_threshold(img_bgr, 0.5),
      l_threshold(img_bgr, 0.9)
    ], 1) * 255.0)


def imshow_u_threshold(img_bgr):
    print('\n===')
    print('Górne odcięcie 0.3 | 0.5 | 0.9\n')
    img_bgr = img_bgr / 255.0
    imshow(np.concatenate([
      u_threshold(img_bgr, 0.3),
      u_threshold(img_bgr, 0.5),
      u_threshold(img_bgr, 0.9)
    ], 1) * 255.0)


def imshow_quad(img_bgr):
    print('\n===')
    print('Funkcja kwadratowa 4.0 | 2.0 | 1.0\n')
    img_bgr = img_bgr / 255.0
    imshow(np.concatenate([
      quad(img_bgr, 4.0),
      quad(img_bgr, 2.0),
      quad(img_bgr, 1.0)
    ], 1) * 255.0)


def imshow_stacked(img_bgr):
    print('\n===')
    print('Złożenie operacji u_thrershold(gamma(invert))) | quad(l_threshold())\n')
    img_bgr = img_bgr / 255.0
    imshow(np.concatenate([
      u_threshold(gamma(invert(img_bgr), 0.3), 0.7),
      quad(l_threshold(img_bgr, 0.4), 3.0)
    ], 1) * 255.0)

### Aplikacja operacji jednopunktowych do obrazów

Powyżej pokazane zostały charakterystyki funkcji realizujących podstawowe operacje na obrazach. Operacje te można bezpośrednio użyć do obrazów, otrzymując nowy, transformowany obraz.

Poniżej zaprezentowane zostały te same funkcje, co powyżej, zastosowane do obrazu Lenna.


In [ ]:
imshow_simple(img)
imshow_gamma(img)
imshow_l_threshold(img)
imshow_u_threshold(img)
imshow_quad(img)
imshow_stacked(img)

# Arytmetyka obrazowa

Wartości intensywności / częstotliwości pikseli obrazów reprezentowane są jako liczby (czy to całkowitoliczbowe czy zmiennoprzecinkowe). Implikuje to możliwość wykonania pewnych operacji na parach (zestawach) obrazów, jak dodawanie, odejmowanie, uśrednianie, itp.

Poniżej zostało przedstawione naiwne rozwiązanie problemu scalania obrazów reprezentujących ten sam obiekt, z ustawioną różną ostrością.

In [ ]:
files = [
    './bug/b_bigbug0000_croppped.png',
    './bug/b_bigbug0001_croppped.png',
    './bug/b_bigbug0002_croppped.png',
    './bug/b_bigbug0003_croppped.png',
    './bug/b_bigbug0004_croppped.png',
    './bug/b_bigbug0005_croppped.png',
    './bug/b_bigbug0006_croppped.png',
    './bug/b_bigbug0007_croppped.png',
    './bug/b_bigbug0008_croppped.png',
    './bug/b_bigbug0009_croppped.png',
    './bug/b_bigbug0010_croppped.png',
    './bug/b_bigbug0011_croppped.png',
    './bug/b_bigbug0012_croppped.png',
]

# wczytanie danych
bugs = [cv2.imread(f, 1) for f in files]
bugs = list(map(lambda i: cv2.resize(i, None, fx=0.3, fy=0.3), bugs))

# wczytanie spodziewanego wyniku algorytmu
result = cv2.imread('./bug/result.png', 1)
result = cv2.resize(result, None, fx=0.3, fy=0.3)

Na wczytanych obrazach możemy wykonać operację uśredniania. Spodziewanym rezultatem jest obraz, z średnio dobrą ostrością.

Gdyby wyszczególnić obszary, w których ostrość obrazu jest wysoka, możliwe byłoby złożenie obrazu ostrego w każdym miejscu (do tego potrzebne są operacje splotowe, które zostaną wprowadzone na kolejnych zajęciach).

In [ ]:
#
bug = np.stack(bugs, 0).mean(0)

print('\n===')
print('Zdjęcia mrówki z ostrością na rożnej odległości\n')
imshow(np.concatenate(bugs[0:4], 1))
imshow(np.concatenate(bugs[4:8], 1))
imshow(np.concatenate(bugs[8:12], 1))
print('\n===')
print('Zwykłe uśrednienie obrazów składowych oraz docelowy obraz\n')
imshow(np.concatenate([bug, result], 1))

### Zadanie 4

Zadanie dotyczy przećwiczenia operacji arytmetycznych na obrazach oraz prostej detekcji cech opartej na przetwarzaniu jednopunktowym.

Korzystając z obrazu lenna znajdź obszary na obrazie, dla których wartości grayscale są z przedziału 120-160.

Następnie zaproponuj operacje, które dla wybranych obszarów zwrócą odwrotność obrazu Lenna (RGB), a dla pozostałych obszarów przekopiują piksele z obrazu Lenna (RGB).


In [ ]:
img_find_val = np.zeros_like(img)
shape = img.shape
for i in range(shape[0]):
    for j in range(shape[1]):
        gray_val = img[i][j][0] * 0.1140 + img[i][j][1] * 0.5870 + img[i][j][2] * 0.2989
        if gray_val >= 120 and gray_val <= 160:
            img_find_val[i][j] = np.full(3, 255) - img[i][j]
        else:
            img_find_val[i][j] = img[i][j]

imshow(img_find_val)

# Operacje geometryczne

Oprócz operacji modyfikujących pojedynczy piksel istnieją również takie, które przekształcają geometrię całego obrazu. Do podstawowych przekształceń geometrycznych zaliczamy:

* przesunięcie (horyzontalne i wertykalne),
* obrót,
* skalowanie,
* pochylenie

Powyższe operacje nazywamy **operacjami affinicznymi** i możemy je reprezentować jako:

$$
T = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}\\
a_{31} & a_{32} & a_{33}
\end{bmatrix}
$$

Wówczas, podstawowe operacje możemy zdefiniować jako:
* tożsamość:
$$
T = \begin{bmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{bmatrix}
$$
* przesunięcie (horyzontalne i wertykalne),
$$
T = \begin{bmatrix}
1 & 0 & t_x\\
0 & 1 & t_y\\
0 & 0 & 1
\end{bmatrix}
$$
* obrót,
$$
T = \begin{bmatrix}
\cos(\alpha) & -\sin(\alpha) & 0\\
\sin(\alpha) & \cos(\alpha) & 0\\
0 & 0 & 1
\end{bmatrix}
$$
* skalowanie,
$$
T = \begin{bmatrix}
c_x & 0 & 0\\
0 & c_y & 0\\
0 & 0 & 1
\end{bmatrix}
$$
* pochylenie
$$
T = \begin{bmatrix}
1 & c_x & 0\\
c_y & 1 & 0\\
0 & 0 & 1
\end{bmatrix}
$$

Powyższe operacje można składać za pomocą mnożenia macierzy.

**Uwaga:**  
OpenCV zawiera operację aplikacji operacji afinicznych, jednak przyjmuje ona przekształcenie w formie:

$$
T = \begin{bmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23}
\end{bmatrix}
$$

In [ ]:
# przesunięcie
t1 = np.array([
  [1, 0, 50],
  [0, 1, -50],
  [0, 0, 1]
], np.float32)

# obrót
t2 = np.array([
  [0.0, -1.0, 256],
  [1.0, 0.0, 0],
  [0, 0, 1]
], np.float32)

# skala
t3 = np.array([
  [0.5, 0, 0],
  [0, 0.5, 0],
  [0, 0, 1]
], np.float32)

img_t1 = cv2.warpAffine(img, t1[:2], img.shape[:2])
img_t2 = cv2.warpAffine(img_t1, t2[:2], img_t1.shape[:2])
img_t3 = cv2.warpAffine(img_t2, t3[:2], img_t2.shape[:2])

imshow(np.concatenate([img, img_t1, img_t2, img_t3], 1))

Na powyższym przykładzie widać, że poszczególne wyniki operacji afinicznych są **stratne**. Można zauważyć, że po pierwszej operacji przesunięcia część kolorowego obrazu znajduje się poza kadrem i zostaje utracona. Skutkuje to błędnym późniejszym przetwarzaniem (mimo poprawnej składni matematycznej).

Rozwiązaniem jest złożenie operacji afinicznych korzystając z operacji mnożenia macierzy. Poniżej znajduje się pojedyncze przekształcenie zawierające wszystkie powyższe operacje, jednocześnie nie tracąc informacji pomiędzy operacjami.

In [ ]:
T = t3 @ t2 @ t1

img_direct = cv2.warpAffine(img, T[:2], img.shape[:2])
imshow(img_direct)

# Histogram


Wyliczenie histogramu polega na zliczeniu liczebności pikseli o danej wartości. Innymi słowy, histogram pokazuje, jak dużo pikseli o pewnej intensywności znajduje się na obrazie.

Aby wyliczyć histogram dla obrazu, możemy skorzystać z gotowej funkcji zawartej w bibliotece matplotlib **hist()**.

In [ ]:
h, _, _ = plt.hist(img_grayscale.flatten(), 256, histtype='step')
plt.show()
imshow(img_grayscale)

Dzieki histogramowi obrazu można zauważyć pewne nieregularności w liczbie występowania intensywności pikseli. W przypadku gdy histogram jest **niezrównoważony**, a więc pewne przedziały intensywności dominują na obrazie, możemy zastosować metodę wyrównywania histogramu, aby przekształcony obraz posiadał bardziej wyrównaną liczbę wszystkich intensywności.

Dzięki takiej operacji możemy obrazy bardzo ciemne, na których na pierwszy rzut oka nie widać żadnych punktów charakterystycznych, przekształcić w taki sposób, aby zmiany intensywności pikseli wyszczególniły wcześniej niewidoczne zmiany na obrazie.

**Wyrównywanie histogramu** zaimplementowane jest w bibliotece OpenCV jako **equalizeHist**, która pobiera obraz na wejściu i zwraca obraz po transformacji.

In [ ]:
img_equalized = cv2.equalizeHist(img_grayscale)
plt.hist(img_equalized.flatten(), 255, histtype='step')
plt.show()
imshow(img_equalized)

Aby ręcznie przeprowadzić wyrównywanie histogramu najpierw należy policzyć **dystrybuantę** intensywności pikseli. Dystrybuanta mówi nam, jakie jest prawdopodobieństwo, że wybierając dowolny piksel z obrazu, jego intensywność będzie mniejsza od danej intensywności na dystrybuancie.

Następnym krokiem jest normalizacja otrzymanej dystrybuanty (aby wartości przeciwdziedziny były z zakresu <0, 255>). W ten sposób otrzymaliśmy, wprowadzoną wcześniej na zajęciach, własną przestrzeń barw (lookup table).

Posiadając tę tablicę, standardowo możemy skorzystać z operacji **LUT()**, aby otrzymać obraz po wyrównaniu histogramu.

In [ ]:
cdf_h = np.cumsum(h)
plt.plot(cdf_h)
cdf_lut = (255 * cdf_h / np.max(cdf_h)).astype(np.uint8)
print(cdf_lut)
imshow(cv2.LUT(img_grayscale, cdf_lut))